In [13]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal, XSD, RDF, RDFS
from rdflib.namespace import RDF, RDFS, OWL

# Создаем граф RDF
g = Graph()

# Определяем пространства имен
BASE = Namespace("http://example.org/skyrim#")
g.bind("base", BASE)

# Абстрактные категории
categories = ["Combat", "Magic", "Crafting", "Abilities"]
for category in categories:
    category_uri = URIRef(BASE[category])
    g.add((category_uri, RDF.type, BASE.Category))

# Навыки и их категории
skills = {
    "Two-Handed": "Combat",
    "One-Handed": "Combat",
    "Archery": "Combat",
    "Destruction": "Magic",
    "Conjuration": "Magic",
    "Alchemy": "Crafting",
    "Smithing": "Crafting"
}
for skill, category in skills.items():
    skill_uri = URIRef(BASE[skill])
    category_uri = URIRef(BASE[category])
    g.add((skill_uri, RDF.type, BASE.Skill))
    g.add((skill_uri, BASE.belongsTo, category_uri))

# Связь персонажей с навыками
characters_skills = {
    "Dovahkiin": ["Two-Handed", "Destruction", "Alchemy"],
    "Ulfric Stormcloak": ["One-Handed", "Smithing"],
    "Farengar": ["Conjuration", "Destruction"]
}
for char, char_skills in characters_skills.items():
    char_uri = URIRef(BASE[char.replace(" ", "_")])
    g.add((char_uri, RDF.type, BASE.Character))
    for skill in char_skills:
        skill_uri = URIRef(BASE[skill])
        g.add((char_uri, BASE.usesSkill, skill_uri))

# Определяем базовые классы и их иерархию
class_hierarchy = {
    "Entity": {
        "Living": {
            "Character": {
                "NPC": {
                    "Merchant": {"Trader", "Innkeeper", "Blacksmith"},
                    "Warrior": {"Soldier", "Guard", "Mercenary"},
                    "Mage": {"Wizard", "Priest", "Conjurer"},
                },
                "Player": set()
            },
            "Creature": {
                "Dragon": set(),
                "Animal": {"Wolf", "Bear", "Mammoth"},
                "Undead": {"Draugr", "Skeleton", "Vampire"}
            }
        },
        "Object": {
            "Item": {
                "Weapon": {"MeleeWeapon", "RangedWeapon", "MagicWeapon"},
                "Armor": {"LightArmor", "HeavyArmor"},
                "Consumable": {"Potion", "Food", "Scroll"},
                "Book": {"SpellTome", "SkillBook", "QuestBook"}
            },
            "Structure": {
                "Building": {"House", "Shop", "Temple"},
                "Location": {"City", "Village", "Cave", "Ruin"}
            }
        },
        "Abstract": {
            "Skill": {
                "Combat": {"OneHanded", "TwoHanded", "Archery"},
                "Magic": {"Destruction", "Restoration", "Conjuration"},
                "Stealth": {"Sneak", "Pickpocket", "Lockpicking"},
                "Crafting": {"Smithing", "Alchemy", "Enchanting"}
            },
            "Faction": {
                "Guild": {"ThievesGuild", "CompanionsGuild", "MagesGuild"},
                "Political": {"Empire", "Stormcloaks"}
            }
        }
    }
}

def add_class_hierarchy(g, hierarchy, parent=None):
    for class_name, subclasses in hierarchy.items():
        class_uri = URIRef(BASE[class_name])
        g.add((class_uri, RDF.type, OWL.Class))

        if parent:
            g.add((class_uri, RDFS.subClassOf, parent))

        if isinstance(subclasses, dict):
            add_class_hierarchy(g, subclasses, class_uri)
        else:  # set
            for subclass in subclasses:
                subclass_uri = URIRef(BASE[subclass])
                g.add((subclass_uri, RDF.type, OWL.Class))
                g.add((subclass_uri, RDFS.subClassOf, class_uri))

# Добавляем иерархию классов в граф
add_class_hierarchy(g, class_hierarchy)

# Определяем свойства
data_properties = {
    "relatedSkill": BASE.relatedSkill,
    "requiresSkill": BASE.requiresSkill,
    "madeOf": BASE.madeOf,
    "cost": BASE.cost,
    "damage": BASE.damage,
    "weight": BASE.weight,
}
obj_properties = {
    "hasSkill": BASE.hasSkill,
    "usesWeapon": BASE.usesWeapon,
    "isMemberOf": BASE.isMemberOf,
    "locatedAt": BASE.locatedAt,
    "authoredBy": BASE.authoredBy,
    "isFriendOf": BASE.isFriendOf,
    "hasRace": BASE.hasRace

}

for prop in data_properties.values():
    g.add((prop, RDF.type, OWL.DatatypeProperty))
for prop in obj_properties.values():
    g.add((prop, RDF.type, OWL.ObjectProperty))

# Указываем характеристики для свойств
g.add((BASE.cost, RDFS.domain, BASE.Weapon))
g.add((BASE.cost, RDFS.range, XSD.string))

g.add((BASE.weight, RDFS.domain, BASE.Weapon))
g.add((BASE.weight, RDFS.range, XSD.string))

g.add((BASE.damage, RDFS.domain, BASE.Weapon))
g.add((BASE.damage, RDFS.range, XSD.string))

g.add((BASE.madeOf, RDFS.domain, BASE.Weapon))
g.add((BASE.madeOf, RDFS.range, XSD.string))

# relatedSkill
g.add((BASE.relatedSkill, RDFS.domain, BASE.Book))
g.add((BASE.relatedSkill, RDFS.range, XSD.boolean))

#requiresSkill
g.add((BASE.requiresSkill, RDFS.domain, BASE.Weapon))
g.add((BASE.requiresSkill, RDFS.range, XSD.string))

# hasSkill - Функциональное свойство, Domain - Character, Range - Skill
g.add((BASE.hasSkill, RDF.type, OWL.FunctionalProperty))
g.add((BASE.hasSkill, RDFS.domain, BASE.Character))
g.add((BASE.hasSkill, RDFS.range, BASE.Skill))

# isMemberOf - не симметричное свойство, Domain - Character, Range - Faction
g.add((BASE.isMemberOf, RDFS.domain, BASE.Character))
g.add((BASE.isMemberOf, RDFS.range, BASE.Faction))

# usesWeapon - не транзитивное свойство, Domain - Character, Range - Weapon
g.add((BASE.usesWeapon, RDFS.domain, BASE.Character))
g.add((BASE.usesWeapon, RDFS.range, BASE.Weapon))

# locatedAt - Функциональное свойство, Domain - Character, Range - Location
g.add((BASE.locatedAt, RDF.type, OWL.FunctionalProperty))
g.add((BASE.locatedAt, RDFS.domain, BASE.Character))
g.add((BASE.locatedAt, RDFS.range, BASE.Location))

# authoredBy - не симметричное свойство, Domain - Book, Range - Character
g.add((BASE.authoredBy, RDFS.domain, BASE.Book))
g.add((BASE.authoredBy, RDFS.range, BASE.Character))

# Добавляем свойство isFriendOf как транзитивное
g.add((BASE.isFriendOf, RDF.type, OWL.TransitiveProperty))  # Указание, что это транзитивное свойство
g.add((BASE.isFriendOf, RDFS.domain, BASE.Character))
g.add((BASE.isFriendOf, RDFS.range, BASE.Character))

# Города и фракции
locations = ["Whiterun", "Windhelm", "Solitude"]
factions = ["Stormcloaks", "Imperial Legion"]
for location in locations:
    loc_uri = URIRef(BASE[location])
    g.add((loc_uri, RDF.type, BASE.City))

for faction in factions:
    faction_uri = URIRef(BASE[faction.replace(" ", "_")])
    g.add((faction_uri, RDF.type, BASE.Faction))

# Связь персонажей с городами и фракциями
g.add((URIRef(BASE["Dovahkiin"]), BASE.residesIn, URIRef(BASE["Whiterun"])))
g.add((URIRef(BASE["Ulfric_Stormcloak"]), BASE.leads, URIRef(BASE["Stormcloaks"])))

# Связь между фракциями
g.add((URIRef(BASE["Stormcloaks"]), BASE.hasEnemy, URIRef(BASE["Imperial_Legion"])))

# Связь магии с расами
races = ["Nord", "Breton", "Dark Elf"]
for race in races:
    race_uri = URIRef(BASE[race.replace(" ", "_")])
    g.add((race_uri, RDF.type, BASE.Race))
g.add((URIRef(BASE["Nord"]), BASE.bonusTo, URIRef(BASE["Two-Handed"])))
g.add((URIRef(BASE["Breton"]), BASE.bonusTo, URIRef(BASE["Conjuration"])))

# Добавляем характеристики персонажей
character_attributes = {
    "level": BASE.hasLevel,
    "magicka": BASE.hasMagicka,
    "stamina": BASE.hasStamina,
    "carry_weight": BASE.hasCarryWeight,
    "armor_rating": BASE.hasArmorRating
}

for prop in character_attributes.values():
    g.add((prop, RDF.type, OWL.DatatypeProperty))
    g.add((prop, RDFS.domain, BASE.Character))
    g.add((prop, RDFS.range, XSD.integer))

# Загрузка данных из CSV
books_df = pd.read_csv("data/skyrim_books.csv")
characters_df = pd.read_csv("data/Skyrim_Named_Characters.csv")
weapons_df = pd.read_csv("data/Skyrim_Weapons.csv")
skyrimstats_df = pd.read_csv("data/skyrimstats.csv")


# Добавляем книги в граф
for _, row in books_df.iterrows():
    book_uri = URIRef(BASE[row["Title"].replace(" ", "_")])
    g.add((book_uri, RDF.type, BASE.Book))
    g.add((book_uri, BASE.cost, Literal(row["Cost"])))
    g.add((book_uri, BASE.description, Literal(row["Description"])))

    if pd.notna(row["Author"]):
        author_uri = URIRef(BASE[str(row["Author"]).replace(" ", "_")])
        # g.add((author_uri, RDF.type, OWL.ObjectProperty))
        g.add((author_uri, RDF.type, BASE.Character))
        g.add((book_uri, obj_properties["authoredBy"], author_uri))

    if row["Skill"] == 1:  # Если Skill равно True, то книга дает навык
        g.add((book_uri, data_properties["relatedSkill"], Literal(True)))
    else:
        g.add((book_uri, data_properties["relatedSkill"], Literal(False)))

# Добавляем персонажей в граф
for _, row in characters_df.iterrows():
    character_uri = URIRef(BASE[row["Name"].replace(" ", "_")])
    # g.add((character_uri, RDF.type, OWL.ObjectProperty))
    g.add((character_uri, RDF.type, BASE.Character))

    if pd.notna(row["Health"]):
        g.add((character_uri, BASE.hasHealth, Literal(row["Health"], datatype=XSD.string))) ####
    if pd.notna(row["Aggression"]):
        g.add((character_uri, BASE.hasAggression, Literal(row["Aggression"], datatype=XSD.string)))

    if pd.notna(row["Race"]):
        race_uri = URIRef(BASE[row["Race"].replace(" ", "_")])
        g.add((race_uri, RDF.type, BASE.Race))
        g.add((character_uri, BASE.hasRace, race_uri))

    if pd.notna(row["Location"]):
        location_uri = URIRef(BASE[row["Location"].replace(" ", "_")])
        g.add((location_uri, RDF.type, BASE.Location))
        g.add((character_uri, obj_properties["locatedAt"], location_uri))

    if pd.notna(row["Faction(s)"]):
        factions = row["Faction(s)"].split("; ")
        for faction in factions:
            faction_uri = URIRef(BASE[faction.replace(" ", "_")])
            g.add((faction_uri, RDF.type, BASE.Faction))
            g.add((character_uri, BASE.isMemberOf, faction_uri))

        for faction in factions:
            faction_uri = URIRef(BASE[faction.replace(" ", "_")])

            # # Ищем всех других персонажей, которые принадлежат этой же фракции
            # for _, other_row in characters_df.iterrows():
            #     if other_row["Name"] != row["Name"] and pd.notna(other_row["Faction(s)"]):
            #         other_factions = other_row["Faction(s)"].split("; ")
            #         if faction in other_factions:
            #             other_character_uri = URIRef(BASE[other_row["Name"].replace(" ", "_")])
            #             # Добавляем связь "isFriendOf"
            #             g.add((character_uri, BASE.isFriendOf, other_character_uri))
            #             g.add((other_character_uri, BASE.isFriendOf, character_uri))

    if pd.notna(row["Primary Skills"]):
        skills = row["Primary Skills"].split(", ")
        for skill in skills:
            skill_uri = URIRef(BASE[skill.replace(" ", "_")])
            g.add((skill_uri, RDF.type, BASE.Skill))
            g.add((character_uri, obj_properties["hasSkill"], skill_uri))

    if pd.notna(row["House"]):
        house_uri = URIRef(BASE[row["House"].replace(" ", "_")])
        g.add((house_uri, RDF.type, BASE.House))
        g.add((character_uri, BASE.livesIn, house_uri))

    if pd.notna(row["Home City"]):
        city_uri = URIRef(BASE[row["Home City"].replace(" ", "_")])
        g.add((city_uri, RDF.type, BASE.City))
        g.add((character_uri, BASE.locatedIn, city_uri))

# Добавляем оружие в граф
for _, row in weapons_df.iterrows():
    weapon_uri = URIRef(BASE[row["Name"].replace(" ", "_")])
    g.add((weapon_uri, RDF.type, BASE.Weapon))
    g.add((weapon_uri, BASE.cost, Literal(row["Gold"])))
    g.add((weapon_uri, BASE.damage, Literal(row["Damage"])))
    g.add((weapon_uri, BASE.weight, Literal(row["Weight"])))
    # Свойство madeOf (материал оружия)
    if pd.notna(row["Upgrade"]):
        # material_uri = URIRef(BASE[row["Type"].replace(" ", "_")])
        # g.add((material_uri, RDF.type, RDFS.Literal))
        g.add((weapon_uri, BASE.madeOf, Literal(row["Upgrade"])))

    # Свойство requiresSkill (необходимый навык для использования оружия)
    if pd.notna(row["Category"]):
        # skill_uri = URIRef(BASE[row["Category"].replace(" ", "_")])
        # g.add((skill_uri, RDF.type, BASE.Skill))
        g.add((weapon_uri, data_properties["requiresSkill"], Literal(row["Category"])))

    if pd.notna(row["Speed"]):
        g.add((weapon_uri, BASE.hasSpeed, Literal(row["Speed"], datatype=XSD.float)))

    if pd.notna(row["Type"]):
        type_uri = URIRef(BASE[row["Type"].replace(" ", "_")])
        g.add((type_uri, RDF.type, BASE.WeaponType))
        g.add((weapon_uri, BASE.hasType, type_uri))


# skills_stats = ["Smithing", "Heavy Armor", "Block", "Two-Handed", "One-Handed", "Archery", "Light"]
# for _, row in skyrimstats_df.iterrows():
#     race_uri = URIRef(BASE[row["Race"].replace(" ", "_")])
#     for skill in skills_stats:
#         if pd.notna(row[skill]) and row[skill] > 0:
#             skill_uri = URIRef(BASE[skill.replace(" ", "_")])
#             g.add((race_uri, BASE.hasSkill, skill_uri))

# Связи между навыками
skill_relations = [
    ("One-Handed", "requires", "Block"),
    ("Two-Handed", "requires", "Heavy_Armor"),
    ("Destruction", "complements", "Conjuration"),
    ("Restoration", "complements", "Alteration"),
    ("Smithing", "enhances", "Heavy_Armor"),
    ("Alchemy", "enhances", "Destruction")
]

for skill1, relation, skill2 in skill_relations:
    skill1_uri = URIRef(BASE[skill1])
    skill2_uri = URIRef(BASE[skill2])
    relation_uri = URIRef(BASE[relation])
    g.add((skill1_uri, relation_uri, skill2_uri))

# Определение квестов
quests = {
    "Main_Quest": ["Dragon_Rising", "The_Way_of_the_Voice"],
    "Companions": ["Proving_Honor", "Glory_of_the_Dead"],
    "College_of_Winterhold": ["First_Lessons", "Under_Saarthal"]
}

for quest_line, quest_names in quests.items():
    quest_line_uri = URIRef(BASE[quest_line])
    g.add((quest_line_uri, RDF.type, BASE.QuestLine))

    for quest in quest_names:
        quest_uri = URIRef(BASE[quest])
        g.add((quest_uri, RDF.type, BASE.Quest))
        g.add((quest_uri, BASE.belongsTo, quest_line_uri))

# Добавление предметов и их свойств
items = {
    "Healing_Potion": {
        "type": "Potion",
        "effect": "Restore_Health",
        "magnitude": 50,
        "duration": 0
    },
    "Enchanted_Ring": {
        "type": "Jewelry",
        "effect": "Fortify_Magicka",
        "magnitude": 20,
        "duration": -1
    }
}

for item_name, properties in items.items():
    item_uri = URIRef(BASE[item_name])
    g.add((item_uri, RDF.type, BASE[properties["type"]]))
    g.add((item_uri, BASE.hasEffect, URIRef(BASE[properties["effect"]])))
    g.add((item_uri, BASE.hasMagnitude, Literal(properties["magnitude"], datatype=XSD.integer)))
    g.add((item_uri, BASE.hasDuration, Literal(properties["duration"], datatype=XSD.integer)))

# Климатические зоны
climate_zones = {
    "Tundra": ["Whiterun", "Dawnstar"],
    "Volcanic": ["Eastmarch", "Riften"],
    "Snowy": ["Winterhold", "Windhelm"]
}

for climate, holds in climate_zones.items():
    climate_uri = URIRef(BASE[climate])
    g.add((climate_uri, RDF.type, BASE.ClimateZone))

    for hold in holds:
        hold_uri = URIRef(BASE[hold])
        g.add((hold_uri, BASE.hasClimate, climate_uri))

# Сохраняем граф в формате Turtle для импорта в Protege
g.serialize("skyrim_knowledge_graph.owl", format="xml")


<Graph identifier=N63fe937da3be420981df2c1faf6dcd68 (<class 'rdflib.graph.Graph'>)>